In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
from random import choice

In [72]:
from random import shuffle

In [5]:
villCentroid = pd.read_csv('SelectedStatesVillagesCentroid.csv')

In [6]:
x_min = min(villCentroid['CentX'])
x_max = max(villCentroid['CentX'])
y_min = min(villCentroid['CentY'])
y_max = max(villCentroid['CentY'])

In [7]:
x_max-x_min

19.764161032105278

In [8]:
y_max - y_min

19.112293241738584

In [9]:
n_r = 75
n_c = 75

In [10]:
x_range = x_max-x_min
y_range = y_max-y_min

In [11]:
### index to fit will be (col*n_r + row) if the 2d matrix is reperesented in 1d list form

In [12]:
villCentroid.head()

,Unnamed: 0,ID,village_code_2011,village_code_2001,CentX,CentY
0,0,BR-0,215990,200.0,83.923688,27.421459
1,1,BR-1,215989,100.0,83.937193,27.437632
2,2,BR-2,215991,300.0,83.875222,27.417117
3,3,BR-3,216180,19200.0,84.111861,27.402037
4,4,BR-4,216179,19100.0,84.093260,27.409085


In [60]:
## creating new id which is unique for all villages 
villCentroid['UID'] = range(len(villCentroid))

In [63]:
cell_length = x_range/((n_c-1)*1.0)
cell_height = y_range/((n_r-1)*1.0)

sorted_villList =  [[] for i in range(n_c*n_r)]
vill_gridNo = [0]*len(villCentroid)
for i in range(len(villCentroid)):
    y = villCentroid['CentY'][i:i+1].values[0].astype(float)
    x = villCentroid['CentX'][i:i+1].values[0].astype(float)
    #vill_code11 = villCentroid['village_code_2011'][i:i+1].values[0].astype(float)
    vill_codeUID = villCentroid['UID'][i:i+1].values[0].astype(float)
    
    c = int((x-x_min)/cell_length)
    r = int((y-y_min)/cell_height)
    
    
    idx = c*n_r + r
    vill_gridNo[i] = idx
    sorted_villList[idx].append(vill_codeUID)

In [106]:
## now lets work on splitting it up in Train:CV:Test
train= 50
cv = 30
test = 20

In [126]:
sparse_vills = []
Train_vills = []
CV_vills = []
Test_vills = []
for vill_list in sorted_villList:
    if len(vill_list) < 10:
        sparse_vills.extend(vill_list)
        continue
        
    shuffle(vill_list)
    lenTrain = int(round(train/100.0*len(vill_list)))
    lenCV = int(round(cv/100.0*len(vill_list)))
    Train_vills.extend(vill_list[:lenTrain])
    CV_vills.extend(vill_list[lenTrain:lenTrain+lenCV])
    Test_vills.extend(vill_list[lenTrain+lenCV:])

## now splitting the sparse_vills list and adding it to train,cv and test sets
lenTrain = int(round(train/100.0*len(sparse_vills)))
lenCV = int(round(cv/100.0*len(sparse_vills)))
Train_vills.extend(sparse_vills[:lenTrain])
CV_vills.extend(sparse_vills[lenTrain:lenTrain+lenCV])
Test_vills.extend(sparse_vills[lenTrain+lenCV:])

In [16]:
df_count = pd.DataFrame(df['grid No'].value_counts())

In [17]:
df_count = df_count.reset_index()

In [18]:
df_count.columns = (['grid no','number_vill'])

In [34]:
len(sorted_villList)

5625

In [24]:
df_count

,grid no,number_vill
0,2525,3212
1,2524,1357
2,2374,897
3,4996,768
4,4998,693
5,4339,674
6,5226,662
7,2599,649
8,4792,625
9,4717,615


In [19]:
df_gridDissolve = df_count.loc[df_count['number_vill']<=10,:]

In [ ]:
df_grid

In [ ]:
df_gridDissolve.head()

In [ ]:
list_gridDissolve = sorted(list(df_gridDissolve['grid no']))

In [ ]:
def possibleLocs(grid_no):
    possible_loc = []
    l_gn = grid_no-1
    r_gn = grid_no+1
    u_gn = grid_no+n_r
    d_gn = grid_no-n_r

    if isValidIdx(l_gn):
        possible_loc.append(l_gn)
    if isValidIdx(r_gn):
        possible_loc.append(r_gn)
    if isValidIdx(u_gn):
        possible_loc.append(u_gn)
    if isValidIdx(d_gn):
        possible_loc.append(d_gn)
    return possible_loc

In [ ]:
def isValidIdx(grid_no):
    if grid_no >= 0 and grid_no < n_c*n_r:
        return True
    return False

In [ ]:
sor_villThreshList = list(sorted_villList)
for grid_no in list_gridDissolve:
    #if len(sor_villThreshList[grid_no]) > 10:
    #    continue
        
    possible_loc = possibleLocs(grid_no)
    new_grid_no = choice(possible_loc)
    print new_grid_no
    sor_villThreshList[new_grid_no].extend(sor_villThreshList[grid_no])